In [2]:
pip install seaborn

  Using cached seaborn-0.12.2-py3-none-any.whl (293 kB)
Note: you may need to restart the kernel to use updated packages.


In [33]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import seaborn as sns
from sklearn.metrics import confusion_matrix 
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer


In [3]:
#################################### PACKAGES ######################################
import pandas as pd
from cleantext import clean
import re

import nltk
from nltk.tokenize import word_tokenize 
from nltk.tokenize import sent_tokenize 
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import *


#R = pd.read_csv("https://raw.githubusercontent.com/several27/FakeNewsCorpus/master/news_sample.csv") 


####################################### CLEANING ######################################
Copy_clean = R.copy()

def cleaningTime(x):
    if isinstance(x, str):
        clean_text = x.replace('|', '')
        clean_text = clean_text.replace('+', '')
        clean_text = clean_text.replace('$', '')
        
        regex = r'\b[xX]+\b'
        regex2 = r'[=]+'
        clean_text = re.sub(regex, '', clean_text)
        clean_text = re.sub(regex2, '', clean_text)
        clean_text = re.sub('<', ' ', clean_text)
        clean_text = re.sub('>', ' ', clean_text)
        
        clean_text = re.sub(r'\d{1,2}\/\d{1,2}\/\d{2,4}', '<DATE>', clean_text)
        
        clean_text = clean(clean_text,
            no_punct=True,
            lower=True,                    # lowercase text
            no_line_breaks=True,           # fully strip line breaks as opposed to only normalizing them
            no_urls=True,                  # replace all URLs with a special token
            no_emails=True,                # replace all email addresses with a special token
            no_numbers=True,
            replace_with_url="<URL>",
            replace_with_email="<EMAIL>",
            replace_with_number="<NUMBER>",
            )
        clean_text = re.sub(r"[^a-zA-Z0-9]+", ' ', clean_text)
        return clean_text
    else:
        return ''

#R['content'].apply(cleaning)
#Copy_clean['content'] = R['content'].apply(cleaningTime)
#print(Copy_clean['content'])
#df = pd.DataFrame(Copy_clean)
#df.to_csv(r'C:\Users\karen\nein.csv', index= None, header=True)


####################################### TOKENIZE #########################################
#Copy_tok = Copy_clean.copy()

def Tokenize(x):     
    tokens = nltk.word_tokenize(x)
    return tokens

#Copy_tok['content'] = Copy_tok['content'].apply(Tokenize)
#print(Copy_tok['content'])


######################################## REMOVING STOPWORDS ################################
#Copy_stop = Copy_tok.copy()
stop_words = set(stopwords.words('english'))
#print("Number of english stopwords: ", len(stop_words))

# removing
def Remove_stopwords(x): 
    filtered_sentence = []
    for w in x:
        if w not in stop_words:
            filtered_sentence.append(w)
    return filtered_sentence

#Copy_stop['content'] = Copy_stop['content'].apply(Remove_stopwords)


######################################### STEMMING ##########################################
#Copy_stemming = Copy_stop.copy()
stemmer = SnowballStemmer("english")

def Stemming_Words(x):
    stem_words = []
    for w in x:
        word = stemmer.stem(w)
        stem_words.append(word)
    return stem_words
#Copy_stemming['content'] = Copy_stemming['content'].apply(Stemming_Words)


######################################## COUNTING WORDS #####################################

def unique_words (file):
    UN = {}
    for row in file:
        for word in row:
            if word not in UN:
                UN[word] = 1
            else: 
                UN[word] += 1
    UN = dict(sorted(UN.items(), key = lambda kv: kv[1], reverse=True)) # Sorting the unique words after number of occurrences, from highest to lowest
    return UN

# number of unique words after...
#print("Number of unique words after tokenization: ", len(unique_words(Copy_tok['content'])))
#print("Number of unique words after removing stopwords: ", len(unique_words(Copy_stop['content'])))
#print("Number of unique words after stemming: ", len(unique_words(Copy_stemming['content'])))

# reductionrates
#print("Reduction of UW from tokenization to after removing stopwords (in %): ", (len(unique_words(Copy_stop['content']))/len(unique_words(Copy_tok['content'])))*100)
#print("Reduction of UW from stopwords to after stemming (in %): ", (len(unique_words(Copy_stemming['content']))/len(unique_words(Copy_stop['content'])))*100)



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\karen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\karen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
df = pd.read_csv('df2_clean_tok_stop_stem.csv', usecols=['content', 'type', 'title'])
df 

C:\Users\karen\AppData\Local\Temp\ipykernel_15184\1582229614.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('df2_clean_tok_stop_stem.csv', usecols=['content', 'type', 'title'])


,type,content,title
0,rumor,"['life', 'illus', 'least', 'quantum', 'level',...",Is life an ILLUSION? Researchers prove 'realit...
1,hate,"['unfortun', 'hasnt', 'yet', 'attack', 'islam'...",Donald Trump
2,hate,"['los', 'angel', 'polic', 'depart', 'deni', 'n...",Donald Trump
3,hate,"['white', 'hous', 'decid', 'quiet', 'withdraw'...","MORE WINNING! Israeli intelligence source, DEB..."
4,hate,"['time', 'come', 'cut', 'tongu', 'support', 'p...","“Oh, Trump, you coward, you just wait, we will..."
...,...,...,...
1999995,unknown,"['emili', 'keeler', 'found', 'curat', 'million...",Emily Keeler
1999996,unknown,"['new', 'inquiri', 'new', 'inquiri', 'space', ...",Chloe Wyma
1999997,bias,"['eric', 'london', 'wide', 'rang', 'nomin', 'l...",Defend Democracy Press
1999998,unknown,"['given', 'choic', 'white', 'black', 'could', ...","In a Mirror, Darkly"


In [12]:
#df.drop('id', inplace=True, axis=1)
#df.drop('inserted_at', inplace=True, axis=1)
#df.drop('scraped_at', inplace=True, axis=1)
#df.drop('updated_at', inplace=True, axis=1)
#df.drop('summary', inplace=True, axis=1)
#df.drop('tags', inplace=True, axis=1)
#df.drop('source', inplace=True, axis=1)

# Remove articles with missing 'type' values
df = df[df['type'].notna()]

# Remove articles with 'type' values = 'unknown'
df = df[df['type'] != 'unknown']

In [13]:
# Print number of duplicates
print("Number of duplicates: ", df.duplicated(subset=['title', 'content']).sum())

Number of duplicates:  196915


In [14]:
# Remove duplicate articles based on 'title' and 'content' columns
df.drop_duplicates(subset=['title', 'content'], inplace=True)

# Print sum of articles for all types
print("Total # of articles: ", df['type'].value_counts().sum())

Total # of articles:  1551222


In [15]:
# Make a new column 'label' and assign 1 to all rows with 'type' = 'fake' or 'conspiracy' or 'junksci' or 'hate' or 'unreliable' or 'rumor' and 0 to all other rows
df['label'] = df['type'].apply(lambda x: 1 if x in ['fake', 'conspiracy', 'junksci', 'hate', 'unreliable', 'rumor'] else 0)

In [16]:
# Split the data into training, validation and test sets
train_set = df.sample(frac=0.8, random_state=0)
new_df2 = df.drop(train_set.index)
validation_set = new_df2.sample(frac=0.5, random_state=0)
test_set = new_df2.drop(validation_set.index)

print(train_set.shape)
print(validation_set.shape)
print(test_set.shape)

(1240978, 4)
(155122, 4)
(155122, 4)


In [ ]:
# Calculate number of unique words in content column
def unique_words (file):
    #return np.unique(file, return_counts=True)
    UN = {}
    for row in file:
        for word in row:
            if word not in UN:
                UN[word] = 1
            else: 
                UN[word] += 1
    #UN = dict(sorted(UN.items(), key = lambda kv: kv[1], reverse=True)) # Sorting the unique words after number of occurrences, from highest to lowest
    return len(UN)
print(type(df['content'][0]))
print("Number of unique words in content column: ", unique_words(df['content'][0]))
print(len(df['content'][0]))

In [ ]:
# Print content of first article in df
print(df['content'][1])
df['content'].shape

In [ ]:
def count_unique (file):
    count = 0
    for word in file:
        count += 1
    return count

print("Number of words in content 0: ", count_unique(df['content'][2]))

In [26]:
#vectorizer = CountVectorizer()

# Fit the vectorizer on the training set
#X_train = vectorizer.fit_transform(train_set['content'])
#y_train = train_set['label']

# Fit the vectorizer on the validation set
#X_val = vectorizer.transform(validation_set['content'])
#y_val = validation_set['label']

# Fit the vectorizer on the test set
#X_test = vectorizer.transform(test_set['content'])
#y_test = test_set['label']

#type(train_set['content'][0])

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\karen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\karen\AppData\Local\Temp\ipykernel_15184\2319731408.py", line 4, in <module>
    X_train = vectorizer.fit_transform(train_set['content'])
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\karen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\feature_extraction\text.py", line 1387, in fit_transform
    vocabulary, X = self._count_vocab(raw_documents, self.fixed_vocabulary_)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\karen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\P

In [30]:
print(train_set['content'])

940911     ['tor', 'tor', 'encrypt', 'anonymis', 'network...
1614902    ['sen', 'graham', 'mccainthi', 'mind', 'specia...
524327     ['yet', 'powersthatb', 'washington', 'wall', '...
214355     ['meico', 'elect', 'pit', 'teacher', 'gang', '...
1886794    ['brock', 'allen', 'turner', 'get', 'even', 's...
                                 ...                        
1493250    ['becom', 'first', 'sit', 'us', 'presid', 'att...
1001369    ['ben', 'hutchison', 'given', 'mansfield', 'le...
1507098    ['presid', 'donald', 'trump', 'made', 'case', ...
1495381    ['presumpt', 'republican', 'nomine', 'donald',...
47988      ['gather', 'number', 'peopl', 'drawn', 'villag...
Name: content, Length: 1240978, dtype: object


In [52]:
print(train_set['content'].shape)

(1240978,)


In [37]:
# # Use TF-IDF to transform the data

#tfidf_transformer = TfidfTransformer()

# Fit the transformer on the training set
#X_train_tfidf = tfidf_transformer.fit_transform(X_train)
y_train = train_set['label']

# Fit the transformer on the validation set
#X_val_tfidf = tfidf_transformer.transform(X_val)
# y_val = validation_set['content']

# Fit the transformer on the test set
#X_test_tfidf = tfidf_transformer.transform(X_test)
# y_test = test_set['label']
X_train_tfidf = TfidfVectorizer(tokenizer=None)
tfidf_matrix = X_train_tfidf.fit_transform(train_set['content'])

In [54]:
print(tfidf_matrix.shape)


(1240978, 1969872)


In [38]:
y_train = train_set['label']

In [40]:
# Train a Naive Bayes classifier with tf-idf data

clf = MultinomialNB().fit(X_train_tfidf, y_train)

# Predict the labels for the validation set
predicted = clf.predict(X_val_tfidf)

# Calculate the accuracy of the model
accuracy_score(y_val, predicted)
print("Accuracy: ", accuracy_score(y_val, predicted))

# Print the classification report
classification_report(y_val, predicted)
print("Classification report: ", classification_report(y_val, predicted))

# Print number of features
print("Number of features: ", clf.n_features_in_)

ValueError: Expected 2D array, got scalar array instead:
array=TfidfVectorizer().
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [50]:
#print the first 10 values in tfidf_matrix
print(tfidf_matrix[0:10])
# Print the shape of tfidf_matrix
print(tfidf_matrix.shape)

  (0, 487130)	0.08858741722494201
  (0, 972426)	0.08798661602748538
  (0, 242189)	0.06786518624582187
  (0, 50686)	0.08885044213547895
  (0, 982885)	0.06690437659219599
  (0, 566072)	0.07244601948288659
  (0, 1491529)	0.07088192583520694
  (0, 1790945)	0.08533985094072333
  (0, 1524381)	0.08492835729998947
  (0, 193711)	0.08575568344228908
  (0, 1607383)	0.08070151332023999
  (0, 392985)	0.16450616932844897
  (0, 1789386)	0.08638738463733635
  (0, 983334)	0.06864194850211712
  (0, 99406)	0.08030076397287096
  (0, 540877)	0.17678843471457514
  (0, 1674797)	0.16796868621101946
  (0, 1847799)	0.17734798518650685
  (0, 976743)	0.07794560220398504
  (0, 305192)	0.05542631501777232
  (0, 1265032)	0.14064463148483033
  (0, 1572785)	0.06597383604199615
  (0, 71078)	0.07652802114098482
  (0, 122450)	0.04714733358798288
  (0, 312917)	0.08389622939067806
  :	:
  (9, 1258793)	0.01471680456187684
  (9, 1530618)	0.024261361556769456
  (9, 990666)	0.02539411286761605
  (9, 1210770)	0.0220097479864128

In [42]:
# Try SVM model on the training set with TF-IDF data

svm = LinearSVC(random_state=0).fit(tfidf_matrix, y_train)

# Predict the labels for the validation set
#predicted = svm.predict(X_val_tfidf)

# Print number of features
#print("Number of features: ", svm.n_features_in_)

# Calculate the accuracy of the model
#accuracy_score(y_val, predicted)
#print("Accuracy: ", accuracy_score(y_val, predicted))

# Print the classification report
#classification_report(y_val, predicted)
#print("Classification report: ", classification_report(y_val, predicted))

In [ ]:
# Train a logistic regression model on the training set
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

LogRegFinal = LogisticRegression(random_state=0, solver='lbfgs', max_iter=2000)

# Fit the model on the training set
LogRegFinal.fit(X_train, y_train)

# Predict the labels of the validation set
y_pred = LogRegFinal.predict(X_val)

# Print the accuracy score
print("Accuracy score: ", accuracy_score(y_val, y_pred))

# Print the classification report
classification_report(y_val, y_pred, output_dict=True)
print("Classification report: ", classification_report(y_val, y_pred))

In [ ]:
# Plot confusion matrix
cm = confusion_matrix(y_val, y_pred)
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.show()

In [ ]:
# Predict the labels of the test set (SVM)
y_pred = svm.predict(X_test_tfidf)

# Print the accuracy score
print("Accuracy score: ", accuracy_score(y_test, y_pred))

# Print the classification report
classification_report(y_test, y_pred)
print("Classification report: ", classification_report(y_test, y_pred))

In [ ]:
# Plot confusion matrix for test set (SVM)
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.show()

# LIAR Dataset

In [20]:
# Load test.tsv file

dftsv = pd.read_csv('test.tsv', sep='\t', header=None)
dftsv.head()

# # # Print statement column for first article
#print(dftsv[2][0])
# print colomn 1 for first 10 articles
print(dftsv[1][0:10])



0           true
1          false
2          false
3      half-true
4     pants-fire
5           true
6           true
7    barely-true
8           true
9    barely-true
Name: 1, dtype: object


In [21]:
# new colomn for the label of the article
dftsv['label'] = dftsv[1].apply(lambda x: 1 if x in ['false', 'pants-fire', 'barely-true'] else 0)
print(dftsv['label'][0:10])

0    0
1    1
2    1
3    0
4    1
5    0
6    0
7    1
8    0
9    1
Name: label, dtype: int64


In [25]:
# cleanin the column 2 of test.tsv file (content) with cleaningTime function
# convert dftsv[2] to string
dftsv[2] = dftsv[2].astype(str)

dftsv[2] = dftsv[2].apply(cleaningTime)
dftsv[2] = dftsv[2].apply(Tokenize)
dftsv[2] = dftsv[2].apply(Remove_stopwords)
dftsv[2] = dftsv[2].apply(Stemming_Words)

#print the shape of the new dftsv


list

In [51]:
print(dftsv[2].shape)


(1267,)


In [59]:
# import TfidfVectorizer
liar_tfidf_matrix = X_train_tfidf.transform(dftsv[2].astype(str))
print(dftsv[2].astype(str))

0       ['build', 'wall', 'usmexico', 'border', 'take'...
1       ['wisconsin', 'pace', 'doubl', 'number', 'layo...
2       ['say', 'john', 'mccain', 'done', 'noth', 'hel...
3       ['suzann', 'bonamici', 'support', 'plan', 'cut...
4       ['ask', 'report', 'whether', 'hes', 'center', ...
                              ...                        
1262    ['say', 'budget', 'provid', 'highest', 'state'...
1263                    ['ive', 'almost', 'everi', 'day']
1264    ['ear', '1980s', 'sen', 'edward', 'kennedi', '...
1265    ['say', 'epa', 'permit', 'languish', 'strickla...
1266    ['say', 'governor', 'go', 'around', 'state', '...
Name: 2, Length: 1267, dtype: object


In [60]:
print(liar_tfidf_matrix.shape)

(1267, 1969872)


In [49]:
print(liar_tfidf_matrix[0:10])
#print the shape of the tfidf_matrix
print(liar_tfidf_matrix.shape)

  (0, 3138)	0.21259570585299392
  (0, 1629)	0.42256852606565126
  (0, 2778)	0.32765069078192804
  (0, 338)	0.37646714852082286
  (0, 2994)	0.46866990361047967
  (0, 3049)	0.39560094897033826
  (0, 377)	0.3885933969714461
  (1, 1579)	0.5501671204976506
  (1, 1931)	0.14357163138878662
  (1, 834)	0.4478202919297405
  (1, 2014)	0.5501671204976506
  (1, 3108)	0.34320166470012897
  (1, 3138)	0.23598531562373068
  (2, 3011)	0.4980273897992678
  (2, 1293)	0.3675709489567165
  (2, 1920)	0.41293489019889473
  (2, 829)	0.4061987069497626
  (2, 1733)	0.35105988528536314
  (2, 1482)	0.3604974074060566
  (2, 2481)	0.1677685298811845
  (3, 2524)	0.3201782965824566
  (3, 65)	0.369699807764814
  (3, 1747)	0.2858917941708913
  (3, 505)	0.3823220812969076
  (3, 691)	0.24059522639927117
  :	:
  (7, 126)	0.1988645618746472
  (7, 61)	0.3398033054095819
  (7, 298)	0.3542665011175018
  (7, 1481)	0.3194185759708568
  (7, 2181)	0.18856853736139134
  (7, 3015)	0.3746512305562269
  (7, 2481)	0.11934036419681605
 

In [61]:
predictions = svm.predict(liar_tfidf_matrix)


In [62]:
#print the accuracy score

print("Accuracy score: ", accuracy_score(dftsv['label'], predictions))

Accuracy score:  0.5493291239147593
